In [41]:
import pandas as pd
import gdown

def read_raw_dataset(type):
  file_id = ''
  if type == 'movies':
    file_id = '1P1q00qlD26wggaJ8oEOFwmeY0q5aulKE'
  elif type == 'tvshows':
    file_id = '1ueHJ-V-TVBsLRWReow-bE5H74EAzm7A3'
  else:
    file_id = '1GHBJ49ZAKQw2N5ry9P_4n802Prx6-tU5'
  dataset_link = f"https://drive.google.com/uc?id={file_id}"
  df = pd.read_csv(dataset_link)
  return df

def read_bert_embeddings(type):
  file_id = ''
  output_file = ''
  if type == 'movies':
    file_id = '1X3ChdISJOgbUJ6WQYDlIVdE40YxkaBb8'
    output_file = 'movie_bert_embeddings_dataframe.h5'
  elif type == 'tvshows':
    file_id = '1yLOOznfWlYeW_R38Wo4RLCTHC2bghnMa'
    output_file = 'tvshows_bert_embeddings_dataframe.h5'
  else:
    file_id = '1dUeZcsrFLVqTp-8EfV4hKCJFKTCDbbf2'
    output_file = 'books_bert_embeddings_dataframe.h5'
  gdown.download(f'https://drive.google.com/uc?id={file_id}', output_file, quiet=False)
  bert_df = pd.read_hdf(output_file)
  return bert_df

In [2]:
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [9]:
def text_to_bert_embedding(doc_text):
    input_ids = tokenizer.encode(doc_text, add_special_tokens=True, return_tensors='pt', padding=True, truncation=True)
    outputs = bert_model(input_ids)
    hidden_states = outputs.last_hidden_state
    return pd.DataFrame(torch.mean(hidden_states, dim=1).detach().numpy())

def refine_input(user_input, user_history):

    # Assign weights
    weight_user_input = 1
    weight_user_history = 1

    # weight_user_input = 0.75
    # weight_user_history = 0.25

    # Apply weights to BERT vectors
    weighted_user_input = user_input * weight_user_input
    weighted_user_history = user_history * weight_user_history

    user_input_bert_embeddings = text_to_bert_embedding(weighted_user_input + weighted_user_history)
    # user_history_bert_embeddings = text_to_bert_embedding(weighted_user_history)

    # Combine weighted BERT vectors
    # combined_bert_vector = user_input_bert_embeddings + user_history_bert_embeddings

    return user_input_bert_embeddings

In [45]:
movies_dataset = read_raw_dataset('movies')
movies_bert_embeddings = read_bert_embeddings('movies')

books_dataset = read_raw_dataset('books')
books_bert_embeddings = read_bert_embeddings('books')

tvshows_dataset = read_raw_dataset('tvshows')
tvshows_bert_embeddings = read_bert_embeddings('tvshows')

# tvshows_dataset = ''
# tvshows_bert_embeddings = ''

# books_dataset = ''
# books_bert_embeddings = ''

Downloading...
From: https://drive.google.com/uc?id=1X3ChdISJOgbUJ6WQYDlIVdE40YxkaBb8
To: /content/movie_bert_embeddings_dataframe.h5
100%|██████████| 139M/139M [00:00<00:00, 153MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dUeZcsrFLVqTp-8EfV4hKCJFKTCDbbf2
To: /content/books_bert_embeddings_dataframe.h5
100%|██████████| 94.2M/94.2M [00:00<00:00, 176MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yLOOznfWlYeW_R38Wo4RLCTHC2bghnMa
To: /content/tvshows_bert_embeddings_dataframe.h5
100%|██████████| 64.3M/64.3M [00:00<00:00, 139MB/s]


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

def get_reccomendations(user_query, media_type):

  if media_type == 'movies':
    raw_dataset = movies_dataset
    bert_embeddings = movies_bert_embeddings
  elif media_type == 'books':
    raw_dataset = books_dataset
    bert_embeddings = books_bert_embeddings
  else:
    raw_dataset = tvshows_dataset
    bert_embeddings = tvshows_bert_embeddings


  sim_scores = list(enumerate(cosine_similarity(bert_embeddings, user_query)))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[0:5]

  top_recommendations = [i[0] for i in sim_scores]
  return top_recommendations

Exception ignored in: <function tqdm.__del__ at 0x7de851be3250>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1149, in __del__
    self.close()
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1278, in close
    if self.last_print_t < self.start_t + self.delay:
AttributeError: 'tqdm' object has no attribute 'last_print_t'


In [ ]:
# API request has 3 params : Media type, User input, User history
# Step 1: Refine user input and user history
# Step 2: Call the appropriate models with refined input
# Step 3: Return results

In [64]:
user_input, user_history = '', ''
user_history = user_input
user_input = 'comedy'
user_query = refine_input(user_input, user_history)

In [65]:
user_query

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.023488,0.108756,-0.455963,0.125545,-0.112093,-0.225967,0.525473,0.191888,-0.334917,0.07941,...,0.437226,-0.240959,0.222678,-0.004011,0.242488,-0.609328,0.03072,0.047155,0.160551,-0.046499


In [66]:
movies_recommendations = get_reccomendations(user_query, 'movies')
books_recommendations = get_reccomendations(user_query, 'books')
tvshows_recommendations = get_reccomendations(user_query, 'tvshows')

In [67]:
result = {'tvshows': [], 'movies':[], 'books': []}

print('Movies')
for idx in movies_recommendations:
  result['movies'].append(movies_dataset.iloc[idx].to_dict())
  print(result['movies'][-1]['title'])

print('\nBooks')
for idx in books_recommendations:
  result['books'].append(books_dataset.iloc[idx].to_dict())
  print(result['books'][-1]['Name'])

print('\nTVShows')
for idc in tvshows_recommendations:
  result['tvshows'].append(tvshows_dataset.iloc[idc].to_dict())
  print(result['tvshows'][-1]['name'])

Movies
El Cementerio de los Elefantes
El violinista del diablo
Ricardo Arjona, Circo Soledad En Vivo
El Ruiseñor de América - Julio Jaramillo
2097

Books
Nonlinear Filtering and Smoothing: An  Introduction to Martingales, Stochastic Integrals and Estimation
Lovely Little Lamb
Planting Patterns: From the Air
The 5-Minute Neurology Consult for PDA: Powered by Skyscape, Inc.
Palaeobiology II

TVShows
Out Of Order
Tenida
Let's Play
Couples
ChuckleMaths


In [51]:
print(result)

{'tvshows': [{'Unnamed: 0': 150535, 'name': 'Out Of Order', 'tags': 'Sketch Comedy TV show Comedy      Benny Brown, Davis Schwartz, , , , ', 'poster': 'https://www.themoviedb.org/t/p/w600_and_h900_bestv2/86Q7ffgkfL82Jr3Zd1hpkG9PRuX.jpg'}, {'Unnamed: 0': 140901, 'name': 'Tenida', 'tags': 'An animation series of comedy. Comedy Animation Drama    zee bangla, , , , , ', 'poster': 'https://www.themoviedb.org/t/p/w600_and_h900_bestv2nan'}, {'Unnamed: 0': 43210, 'name': "Let's Play", 'tags': 'Achievement Hunter employees play video games, usually extremely poorly. Comedy      Geoff Ramsey, Jack Pattillo, , , , ', 'poster': 'https://www.themoviedb.org/t/p/w600_and_h900_bestv2/6Pf6w9ykQPgZdDIG0vL83vVDeeN.jpg'}, {'Unnamed: 0': 55896, 'name': 'Couples', 'tags': 'Couples TV Show Drama      Tony Parker, , , , , ', 'poster': 'https://www.themoviedb.org/t/p/w600_and_h900_bestv2nan'}, {'Unnamed: 0': 76114, 'name': 'ChuckleMaths', 'tags': 'Educational show, Paul and Barry are running a hotel. Comedy Ki